### https://github.com/odegeasslbc/FastGAN-pytorch

In [1]:
import os
import shutil
import pandas as pd
from collections import defaultdict

In [2]:
## Conexão com o google drive, se achar necessário
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!git clone https://github.com/odegeasslbc/FastGAN-pytorch.git

Cloning into 'FastGAN-pytorch'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 122 (delta 6), reused 8 (delta 2), pack-reused 108
Receiving objects: 100% (122/122), 131.87 KiB | 2.23 MiB/s, done.
Resolving deltas: 100% (46/46), done.


In [4]:
!nvidia-smi

Sun Jul  3 01:13:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
#!mkdir "/content/data"
#!wget "http://groups.csail.mit.edu/vision/LabelMe/NewImages/indoorCVPR_09.tar"
#!tar -xvf "/content/indoorCVPR_09.tar" -C "/content/data"
#!mv "/content/data/Images" "/content/data/indoorCVPR_09"

!mkdir "/content/data"
!tar -xvf "/content/drive/MyDrive/indoorCVPR_09.tar" -C "/content/data"
!mv "/content/data/Images" "/content/data/indoorCVPR_09"

Streaming output truncated to the last 5000 lines.
Images/mall/cover_home3.jpg
Images/mall/CP67791.jpg
Images/mall/crocker_galleria_735089.jpg
Images/mall/mall1.jpg
Images/mall/d8.jpg
Images/mall/deira_city_centre_dubai_08.jpg
Images/mall/dungarv_new_mall.jpg
Images/mall/latham_circle_mall_16.jpg
Images/mall/eastridg1.jpg
Images/mall/IMG_6449.jpg
Images/mall/main_boutique_b.jpg
Images/mall/easyfranchise_centre_commercial_1.jpg
Images/mall/ins42.jpg
Images/mall/mall14.jpg
Images/mall/eaton_centre_2001_03.jpg
Images/mall/ins43.jpg
Images/mall/mall17.jpg
Images/mall/full_shopping_Central_1.jpg
Images/mall/galleria.jpg
Images/mall/galleria1.jpg
Images/mall/galleria2.jpg
Images/mall/Galleria_003.jpg
Images/mall/laurel_mall_06.jpg
Images/mall/Galleria_1.jpg
Images/mall/galleria_san_federico_nr_109_2_n1.jpg
Images/mall/ins23.jpg
Images/mall/ins44.jpg
Images/mall/mall18.jpg
Images/mall/galleria_vittorio_emanuele_ii_milan_mlgalar.jpg
Images/mall/ins45.jpg
Images/mall/mall19.jpg
Images/mall/GUM_

!python /content/FastGAN-pytorch/train.py --path /content/data/indoorCVPR_09/bedroom --name fastGan_20220618

In [6]:
def build_folders(in_path, out_path, out_file):
  try: 
    # Ajusta a estrutura de pastas
    files=[x.split("/")[-1]for x in [x[0] for x in os.walk(os.getcwd() + in_path)][1:]]
    files.sort()
    labeldict={}
    count=0

    for file in files:
        labeldict[file]=count
        count+=1
        
    class_freq=defaultdict(int)
    csvlist=[]

    for file in files:
        images=os.listdir(os.path.join(os.getcwd() + in_path,file))
        count=1
        for img in images:
            an=img.split(".")        
            newname=str(file)+"_"+str(count)+"."+str(an[-1])
            os.rename(os.path.join(os.getcwd() + in_path,file,img), os.path.join(os.getcwd() + out_path,newname))
            csvlist.append([file, labeldict[file],newname])
            class_freq[file]+=1
            count+=1
            
    for file in files:
        os.rmdir(os.path.join(os.getcwd() + in_path,file))

    df = pd.DataFrame(csvlist,columns=["Category", "Category_Id","File"]) 
    df.to_csv(out_file)
    return df
  except:
    raise "Error"

In [7]:
in_path = "/data/indoorCVPR_09"
out_path = "/data/indoorCVPR_09"
out_file = "train.csv"
df = build_folders(in_path, out_path, out_file)

#### Seleção de imagens do Helder

In [8]:
selected_images_df = pd.read_csv("/content/drive/MyDrive/Helder/Train.csv", sep = ",").drop("Unnamed: 0", axis = 1)
selected_images = selected_images_df.File.to_list()

In [9]:
print(len(selected_images))

331


In [10]:
for img in os.listdir("/content/data/indoorCVPR_09"):
  if img not in selected_images:
    os.remove(f"/content/data/indoorCVPR_09/{img}")

In [11]:
len(os.listdir("/content/data/indoorCVPR_09"))

331

### Parte I: até 50000

In [ ]:
!python "/content/FastGAN-pytorch/train.py" \
--path "/content/data/indoorCVPR_09" \
--name "fastGan_20220628"

Setting up Perceptual loss...
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100% 528M/528M [00:02<00:00, 200MB/s]
Loading model from: /content/FastGAN-pytorch/lpips/weights/v0.1/vgg.pth
...[net-lin [vgg]] initialized
...Done
Namespace(batch_size=8, ckpt='None', cuda=0, im_size=1024, iter=50000, name='fastGan_20220628', path='/content/data/indoorCVPR_09', start_iter=0)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0% 0/50001 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/functional.py:568:

In [ ]:
!mv "/content/train_results/fastGan_20220628" "/content/drive/MyDrive"

In [ ]:
!python "/content/FastGAN-pytorch/eval.py" --artifacts "/content/drive/MyDrive/fastGan_20220628" --n_sample 662 --start_iter 5 --end_iter 5

load checkpoint success, epoch 50000
100% 41/41 [01:24<00:00,  2.06s/it]


### Parte II: de 50000 até 100000

In [ ]:
!python "/content/FastGAN-pytorch/train.py" \
--path "/content/data/indoorCVPR_09" \
--name "fastGan_20220628_pt2" \
--iter 100000 \
--ckpt "/content/drive/MyDrive/fastGan_20220628/models/all_50000.pth"

Setting up Perceptual loss...
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100% 528M/528M [00:02<00:00, 243MB/s]
Loading model from: /content/FastGAN-pytorch/lpips/weights/v0.1/vgg.pth
...[net-lin [vgg]] initialized
...Done
Namespace(batch_size=8, ckpt='/content/drive/MyDrive/fastGan_20220628/models/all_50000.pth', cuda=0, im_size=1024, iter=100000, name='fastGan_20220628_pt2', path='/content/data/indoorCVPR_09', start_iter=0)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0% 0/50001 [00:00<?, ?it/s]/u

In [ ]:
!mv "/content/train_results/fastGan_20220628_pt2" "/content/drive/MyDrive"

In [ ]:
!python "/content/FastGAN-pytorch/eval.py" --artifacts "/content/drive/MyDrive/fastGan_20220628_pt2" --n_sample 662 --start_iter 10 --end_iter 10

load checkpoint success, epoch 100000
100% 41/41 [01:24<00:00,  2.05s/it]


### Parte III: de 100000 até 150000

In [ ]:
!python "/content/FastGAN-pytorch/train.py" \
--path "/content/data/indoorCVPR_09" \
--name "fastGan_20220628_pt3" \
--iter 150000 \
--ckpt "/content/drive/MyDrive/fastGan_20220628_pt2/models/all_100000.pth"

Setting up Perceptual loss...
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100% 528M/528M [00:06<00:00, 80.3MB/s]
Loading model from: /content/FastGAN-pytorch/lpips/weights/v0.1/vgg.pth
...[net-lin [vgg]] initialized
...Done
Namespace(batch_size=8, ckpt='/content/drive/MyDrive/fastGan_20220628_pt2/models/all_100000.pth', cuda=0, im_size=1024, iter=150000, name='fastGan_20220628_pt3', path='/content/data/indoorCVPR_09', start_iter=0)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
  0% 0/50001 [00:00<?, ?i

In [ ]:
!mv "/content/train_results/fastGan_20220628_pt3" "/content/drive/MyDrive"

In [ ]:
!python "/content/FastGAN-pytorch/eval.py" --artifacts "/content/drive/MyDrive/fastGan_20220628_pt3" --n_sample 662 --start_iter 15 --end_iter 15

load checkpoint success, epoch 150000
100% 41/41 [01:22<00:00,  2.02s/it]


### Parte IV Eval com 10000

In [12]:
!python "/content/FastGAN-pytorch/eval.py" --artifacts "/content/drive/MyDrive/fastGan_20220628_pt3" --n_sample 662 --start_iter 1 --end_iter 1

load checkpoint success, epoch 10000
100% 41/41 [01:24<00:00,  2.06s/it]
